In [1]:
import os
import zipfile
import pandas as pd
import re
import requests

# Разархивация данных

In [189]:
import os
import zipfile

folder_path = r'C:\Users\PC\Desktop\работа\финансы вб\Ридберг'

# Проходимся по всем файлам в папке
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    
    # Проверяем, является ли файл zip-архивом
    if os.path.isfile(file_path) and file_name.endswith(".zip"):
        
        # Извлекаем название архива без расширения
        archive_name = os.path.splitext(file_name)[0]
        
        # Создаем папку с названием архива, если её еще нет
        extract_folder = os.path.join(folder_path, archive_name)
        if not os.path.isdir(extract_folder):
            os.mkdir(extract_folder)
        
        # Разархивируем файлы в папку названия архива
        with zipfile.ZipFile(file_path, "r") as zip_ref:
            zip_ref.extractall(extract_folder)
        
        # Переименовываем файлы в соответствии с названием архива
        for extracted_file in os.listdir(extract_folder):
            old_file_path = os.path.join(extract_folder, extracted_file)
            new_file_name = f"{archive_name}_{extracted_file}"
            new_file_path = os.path.join(extract_folder, new_file_name)
            os.rename(old_file_path, new_file_path)

# Обработка файлов финансовых с вб

In [2]:
import os
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

folder_path = r'C:\Users\PC\Desktop\работа\финансы вб\Ридберг'

# Функция для чтения и обработки одного файла
def process_file(file_path):
    df = pd.read_excel(file_path)
    df['Источник'] = os.path.basename(file_path)
    return df

dfs = []

with ThreadPoolExecutor() as executor:
    for root, _, files in os.walk(folder_path):
        for filename in files:
            if filename.endswith(".xlsx"):
                file_path = os.path.join(root, filename)
                future = executor.submit(process_file, file_path)
                dfs.append(future.result())

combined_df = pd.concat(dfs, ignore_index=True)

In [3]:
#combined_df.head()
combined_df.columns


Index(['№', 'Номер поставки', 'Предмет', 'Код номенклатуры', 'Бренд',
       'Артикул поставщика', 'Название', 'Размер', 'Баркод', 'Тип документа',
       'Обоснование для оплаты', 'Дата заказа покупателем', 'Дата продажи',
       'Кол-во', 'Цена розничная', 'Вайлдберриз реализовал Товар (Пр)',
       'Согласованный продуктовый дисконт, %', 'Промокод %',
       'Итоговая согласованная скидка',
       'Цена розничная с учетом согласованной скидки',
       'Размер снижения кВВ из-за рейтинга, %',
       'Размер снижения кВВ из-за акции, %',
       'Скидка постоянного Покупателя (СПП)', 'Размер кВВ, %',
       'Размер  кВВ без НДС, % Базовый', 'Итоговый кВВ без НДС, %',
       'Вознаграждение с продаж до вычета услуг поверенного, без НДС',
       'Возмещение за выдачу и возврат товаров на ПВЗ',
       'Возмещение издержек по эквайрингу',
       'Вознаграждение Вайлдберриз (ВВ), без НДС',
       'НДС с Вознаграждения Вайлдберриз',
       'К перечислению Продавцу за реализованный Товар', 'К

In [5]:
data_i = combined_df

In [51]:
df_fin_i=combined_df[['Источник', 'Бренд',
        'Код номенклатуры','Обоснование для оплаты','Дата заказа покупателем', 'Дата продажи','Кол-во']].copy()

In [57]:
# Подзадача Александра С
df_fin=combined_df[['Источник', 'Код номенклатуры', 'Артикул поставщика','Тип документа','Обоснование для оплаты', 'ШК','Кол-во',
        'К перечислению Продавцу за реализованный Товар']].copy()
df_fin['Источник'] = df_fin['Источник'].str.replace('Детализация отчета ', '').str.replace('_0.xlsx', '')


C:\Users\PC\AppData\Local\Temp\ipykernel_10248\3077273544.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_fin['Источник'] = df_fin['Источник'].str.replace('Детализация отчета ', '').str.replace('_0.xlsx', '')


In [58]:
df_fin['Итого'] = df_fin['Кол-во']*df_fin['К перечислению Продавцу за реализованный Товар']

In [59]:
df_fin

,Источник,Код номенклатуры,Артикул поставщика,Тип документа,Обоснование для оплаты,ШК,Кол-во,К перечислению Продавцу за реализованный Товар,Итого
0,10133957,40790999,30601621#1194438r,Продажа,Продажа,6005560161,1,8489.49,8489.49
1,10133957,36081507,27092322#1012792r,Продажа,Продажа,5920737469,1,3303.60,3303.60
2,10133957,32348837,24244193#1189130r,Продажа,Продажа,5881568958,1,461.89,461.89
3,10133957,55940833,1857_4#1198560r,Возврат,Возврат,5876965053,1,574.67,574.67
4,10133958,38327157,28817639#1193418r,Продажа,Продажа,6017423906,1,4790.16,4790.16
...,...,...,...,...,...,...,...,...,...
310906,9878644,43399301,32358916#1195628r,NaN,Логистика,5177152741,0,0.00,0.00
310907,9878644,43399301,32358916#1195628r,NaN,Логистика,5177152757,0,0.00,0.00
310908,9878644,43399301,32358916#1195628r,NaN,Логистика,5177152713,0,0.00,0.00
310909,9878644,43399301,32358916#1195628r,NaN,Логистика,5167112252,0,0.00,0.00


In [60]:
df_fin['Источник'] = df_fin['Источник'].str.replace('Детализация отчета ', '').str.replace('_0.xlsx', '')

C:\Users\PC\AppData\Local\Temp\ipykernel_10248\63343660.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_fin['Источник'] = df_fin['Источник'].str.replace('Детализация отчета ', '').str.replace('_0.xlsx', '')


In [61]:
new_df = df_fin.loc[df_fin['Обоснование для оплаты'].isin([ 'Авансовая оплата за товар без движения'])]
new_df.to_excel('itog/sverka_wb_r.xlsx',index=False, encoding = 'utf-8')

# Работа с выгрузкой общей на вб

In [18]:
# Укажите путь к файлу xlsx
file_path = 'WB_r.xlsx'
# Считывание данных из файла xlsx в датафрейм
df = pd.read_excel(file_path)
df.columns

Index(['№ отчета', 'Юридическое лицо', 'Дата начала', 'Дата конца',
       'Дата формирования', 'Продажа', 'Согласованная скидка, %',
       'К перечислению за товар', 'Стоимость логистики',
       'Повышенная логистика согласно коэффициенту по обмерам',
       'Другие виды штрафов', 'Общая сумма штрафов', 'Доплаты',
       'Стоимость хранения', 'Стоимость платной приемки', 'Прочие удержания',
       'Итого к оплате без учета долга предыдущих отчетов', 'Итого к оплате',
       'Валюта', 'Статус отчета'],
      dtype='object')

In [19]:
df_WB_a = df[['№ отчета','Дата начала']]
df_WB_a.head()

,№ отчета,Дата начала
0,47151829,2023-10-30T00:00:00Z
1,47151828,2023-10-30T00:00:00Z
2,46585898,2023-10-23T00:00:00Z
3,46585897,2023-10-23T00:00:00Z
4,46475114,2023-10-16T00:00:00Z


# Обработка 1С

In [7]:
# Укажите путь к файлу xlsx
file_path = r'C:\Users\PC\Downloads\1cwb.xlsx'
# Считывание данных из файла xlsx в датафрейм
df = pd.read_excel(file_path)
df_1C_a = df[['Артикул','Номенклатура','Количество']]
df_1C_a = df_1C_a.dropna(subset=['Количество'])
data_list = df_1C_a.values.tolist()
future_df=[]
name = ["" , ""]
for i in data_list:
    if "Неделя с" not in i[1]:
        name = i[0:2]
    else:
        list1 = []
        list1.extend(name)
        list1.extend(i[1:3])
        future_df.append(list1)
df_1C_a = pd.DataFrame(future_df, columns=['Артикул','Номенклатура','Дата','Количество'])
df_1C_a['Количество'].sum()

81978

In [8]:
df_1C_a.to_excel("new_1с.xlsx", index=False)

# Обработка файлов сопоставления

In [21]:
# Укажите путь к файлу xlsx
file_path = 'Айковер.xlsx'
# Считывание данных из файла xlsx в датафрейм
df = pd.read_excel(file_path)
df_art_a = df[['Артикул', 'ВБ_Артикул']].copy()
df_art_a

,Артикул,ВБ_Артикул
0,1002812,39902779
1,1002821,39902757
2,1002901,44042088
3,1004073,51384667
4,1004104,46174079
...,...,...
5671,1209400,139613863
5672,1209401,139613862
5673,1209412,139344960
5674,1209413,139348647


# Объединение источников

In [22]:
#Добавление к финансовому отчету артикулов из нулевого
df_fin_i = df_fin_i.merge(df_art_a, left_on='Код номенклатуры', right_on='ВБ_Артикул', how='left')

In [23]:
#Добавление к финансовому отчету дат периодов из общих фин выгрузок
df_fin_i['Источник'] = df_fin_i['Источник'].astype(int)
df_fin_i = df_fin_i.merge(df_WB_a, left_on='Источник', right_on='№ отчета', how='left')

In [24]:
df_fin_i.head()

,Источник,Бренд,Код номенклатуры,Обоснование для оплаты,Дата заказа покупателем,Дата продажи,Кол-во,Артикул,ВБ_Артикул,№ отчета,Дата начала
0,10133957,Бюрократ,40790999,Продажа,2022-05-04,2022-05-11,1,1194438,40790999.0,10133957,2022-05-09T00:00:00Z
1,10133957,CHAIRMAN,36081507,Продажа,2022-05-05,2022-05-13,1,1012792,36081507.0,10133957,2022-05-09T00:00:00Z
2,10133957,Karcher,32348837,Продажа,2022-05-06,2022-05-09,1,1189130,32348837.0,10133957,2022-05-09T00:00:00Z
3,10133957,WiWU,55940833,Возврат,2022-04-20,2022-04-27,1,1198560,55940833.0,10133957,2022-05-09T00:00:00Z
4,10133958,Яндекс,38327157,Продажа,2022-05-05,2022-05-10,1,1193418,38327157.0,10133958,2022-05-09T00:00:00Z


In [25]:

df_end = df_fin_i[['Источник','Бренд','Код номенклатуры','Обоснование для оплаты', 'Артикул', 'Дата начала','Кол-во' ]]

In [26]:
df_end = df_end.loc[df_end['Обоснование для оплаты'].isin(['Продажа', 'Корректная продажа', 'Сторно продаж','Возврат','Сторно возвратов','Корректный возврат'])]

In [27]:
df_end

,Источник,Бренд,Код номенклатуры,Обоснование для оплаты,Артикул,Дата начала,Кол-во
0,10133957,Бюрократ,40790999,Продажа,1194438,2022-05-09T00:00:00Z,1
1,10133957,CHAIRMAN,36081507,Продажа,1012792,2022-05-09T00:00:00Z,1
2,10133957,Karcher,32348837,Продажа,1189130,2022-05-09T00:00:00Z,1
3,10133957,WiWU,55940833,Возврат,1198560,2022-05-09T00:00:00Z,1
4,10133958,Яндекс,38327157,Продажа,1193418,2022-05-09T00:00:00Z,1
...,...,...,...,...,...,...,...
335268,9878644,CHAIRMAN,36081507,Продажа,1012792,2022-05-02T00:00:00Z,1
335270,9878644,CHAIRMAN,36081507,Продажа,1012792,2022-05-02T00:00:00Z,1
335275,9878644,CHAIRMAN,36081507,Продажа,1012792,2022-05-02T00:00:00Z,1
335276,9878644,CHAIRMAN,36081507,Продажа,1012792,2022-05-02T00:00:00Z,1


In [29]:
# Создание пустого нового датафрейма
new_df = df_fin_i.loc[df_fin_i['Обоснование для оплаты'].isin(['Логистика', 'Штрафы и доплаты', 'Штрафы', 'Частичная компенсация брака', 'Логистика сторно', 'Доплаты', 'Компенсация доставки до клиента', 'Авансовая оплата за товар без движения', 'Возмещение издержек по перевозке', 'Компенсация подмененного товара'])]
new_df 

,Источник,Бренд,Код номенклатуры,Обоснование для оплаты,Дата заказа покупателем,Дата продажи,Кол-во,Артикул,ВБ_Артикул,№ отчета,Дата начала
6,10133958,Бюрократ,42292499,Логистика,2022-05-10,2022-05-10,0,1129679,42292499.0,10133958,2022-05-09T00:00:00Z
8,10133958,Nika,62316840,Логистика,2022-05-10,2022-05-10,0,1201106,62316840.0,10133958,2022-05-09T00:00:00Z
9,10133958,Nika,62316840,Логистика,2022-05-10,2022-05-10,0,1201106,62316840.0,10133958,2022-05-09T00:00:00Z
10,10133958,Deerma,68983695,Логистика,2022-05-10,2022-05-10,0,1202772,68983695.0,10133958,2022-05-09T00:00:00Z
11,10133958,HyperX,43881009,Логистика,2022-05-10,2022-05-10,0,1187866,43881009.0,10133958,2022-05-09T00:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...
335300,9878644,Deerma,43399301,Логистика,2022-05-06,2022-05-06,0,1195628,43399301.0,9878644,2022-05-02T00:00:00Z
335301,9878644,Deerma,43399301,Логистика,2022-05-06,2022-05-06,0,1195628,43399301.0,9878644,2022-05-02T00:00:00Z
335302,9878644,Deerma,43399301,Логистика,2022-05-06,2022-05-06,0,1195628,43399301.0,9878644,2022-05-02T00:00:00Z
335303,9878644,Deerma,43399301,Логистика,2022-05-06,2022-05-06,0,1195628,43399301.0,9878644,2022-05-02T00:00:00Z


In [30]:
# Преобразование строки в формат datetime
new_df['Дата начала'] = pd.to_datetime(new_df['Дата начала'])
# Преобразование формата даты
new_df['Дата начала'] = new_df['Дата начала'].dt.strftime('%d.%m.%Y')

C:\Users\PC\AppData\Local\Temp\ipykernel_14768\844454320.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Дата начала'] = pd.to_datetime(new_df['Дата начала'])
C:\Users\PC\AppData\Local\Temp\ipykernel_14768\844454320.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Дата начала'] = new_df['Дата начала'].dt.strftime('%d.%m.%Y')


In [31]:
# Преобразование строки в формат datetime
df_end['Дата начала'] = pd.to_datetime(df_end['Дата начала'])
# Преобразование формата даты
df_end['Дата начала'] = df_end['Дата начала'].dt.strftime('%d.%m.%Y')

In [32]:
df_end = df_end.groupby(['Источник', 'Бренд', 'Код номенклатуры', 'Обоснование для оплаты',
       'Артикул', 'Дата начала'])['Кол-во'].sum().reset_index()
df_end

,Источник,Бренд,Код номенклатуры,Обоснование для оплаты,Артикул,Дата начала,Кол-во
0,3278313,PitEco,28746542,Продажа,gru1629,24.05.2021,2
1,3278313,RIDBERG,28497873,Продажа,1211603,24.05.2021,6
2,3278313,RIDBERG,28747939,Продажа,1129682,24.05.2021,4
3,3278313,RIDBERG,28747939,Продажа,1205865,24.05.2021,4
4,3278313,Razer,28490840,Продажа,1193035,24.05.2021,1
...,...,...,...,...,...,...,...
28403,46475114,Бюрократ,154557194,Корректный возврат,1211234,16.10.2023,1
28404,46475114,Бюрократ,154557194,Продажа,1211234,16.10.2023,11
28405,46475114,Бюрократ,154557194,Сторно возвратов,1211234,16.10.2023,1
28406,46475114,Бюрократ,154557194,Сторно продаж,1211234,16.10.2023,8


# Выгрузка в Excel

In [33]:
df_end.to_excel('itog/wb_r.xlsx',index=False, encoding = 'utf-8')

In [12]:
df_1C_a['Дата'] = df_1C_a['Дата'].str.replace('Неделя с ', '')


In [13]:
df_1C_a.to_excel('itog/1c_r.xlsx',index=False, encoding = 'utf-8')

# Выгрузка по api WB

In [35]:
import requests
from requests.auth import HTTPBasicAuth

In [38]:
BASE_URL = 'https://statistics-api.wildberries.ru/api/v1/supplier/reportDetailByPeriod'
response = requests.get(f"{BASE_URL}/products", )

In [39]:
print(response.json())

{'code': 401, 'message': '07e4668e-a53a3d31f8b0-UK:P85hcWze2KF unauthorized, rejected; 4760933a-29 empty Authorization header'}


In [ ]:
method = "get"
url = 'https://statistics-api.wildberries.ru/api/v1/supplier/reportDetailByPeriod'
apiKey = ''
auth = HTTPBasicAuth(apiKey, secret)
rsp = requests.request(method, url, headers=None, auth=auth)